In [1]:
from qiskit import Aer
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.drivers.second_quantization import (
    ElectronicStructureDriverType,
    ElectronicStructureMoleculeDriver,
)
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

molecule = Molecule(
    geometry=[["H", [0.0, 0.0, 0.0]], ["H", [0.0, 0.0, 0.735]]], charge=0, multiplicity=1
)
driver = ElectronicStructureMoleculeDriver(
    molecule, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF
)

es_problem = ElectronicStructureProblem(driver) #should be replaced by ising model
qubit_converter = QubitConverter(JordanWignerMapper()) #coverts fermions to spins

Numpy 1.16 has memory leak bug  https://github.com/numpy/numpy/issues/13808
It is recommended to downgrade to numpy 1.15 or older


In [2]:
from qiskit.algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()

In [3]:
from qiskit.providers.aer import StatevectorSimulator
from qiskit import Aer
from qiskit.utils import QuantumInstance
from qiskit_nature.algorithms import VQEUCCFactory

quantum_instance = QuantumInstance(backend=Aer.get_backend("aer_simulator_statevector"))
vqe_solver = VQEUCCFactory(quantum_instance)

In [11]:
from qiskit.algorithms import VQE
from qiskit.circuit.library import TwoLocal

tl_circuit = TwoLocal(
    rotation_blocks=["h", "rx"],
    entanglement_blocks="cz",
    entanglement="full",
    reps=2,
    parameter_prefix="y",
)

another_solver = VQE(
    ansatz=tl_circuit,
    quantum_instance=QuantumInstance(Aer.get_backend("aer_simulator_statevector")),
)

print(es_problem.second_q_ops())

[FermionicOp([('+_0 -_1 +_2 -_3', (0.18093119978423128+0j)), ('+_0 -_1 -_2 +_3', (-0.18093119978423128+0j)), ('-_0 +_1 +_2 -_3', (-0.18093119978423128+0j)), ('-_0 +_1 -_2 +_3', (0.18093119978423136+0j)), ('+_...)], register_length=4, display_format='sparse'), FermionicOp([('+_0 -_0', (1+0j)), ('+_1 -_1', (1+0j)), ('+_2 -_2', (1+0j)), ('+_3 -_3', (1+0j))], register_length=4, display_format='sparse'), FermionicOp([('+_0 -_1 -_2 +_3', (1+0j)), ('-_0 +_1 +_2 -_3', (1+0j)), ('+_3 -_3', (0.75+0j)), ('+_2 -_2', (0.75+0j)), ('+_2 -_2 +_3 -_3', (0.5+0j)), ('+_1 -_1', (0.75+0j)), ('+_1 -_1 +_3 -_3', (-1.5+0j)), (...)], register_length=4, display_format='sparse'), FermionicOp([('+_0 -_0', (0.5+0j)), ('+_1 -_1', (0.5+0j)), ('+_2 -_2', (-0.5+0j)), ('+_3 -_3', (-0.5+0j))], register_length=4, display_format='sparse'), FermionicOp([('', 0j)], register_length=4, display_format='sparse'), FermionicOp([('', 0j)], register_length=4, display_format='sparse'), FermionicOp([('+_0 -_1', (-0.9278334704592319+0

In [5]:
from qiskit_nature.algorithms import GroundStateEigensolver

calc = GroundStateEigensolver(qubit_converter, vqe_solver)
res = calc.solve(es_problem)

print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030144
  - computed part:      -1.857275030144
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035695
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: -0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.38894847]
    - computed part:      [0.0  0.0  1.38894847]
  > Dipole moment (a.u.): [0.0  0.0  0.00000023]  Total: 0.00000023
                 (debye): [0.0  0.0  0.0000006]  Total: 0.0000006
 


In [6]:
calc = GroundStateEigensolver(qubit_converter, numpy_solver)
res = calc.solve(es_problem)
print(res)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275030202
  - computed part:      -1.857275030202
~ Nuclear repulsion energy (Hartree): 0.719968994449
> Total ground state energy (Hartree): -1.137306035753
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.3889487]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.3889487]
    - computed part:      [0.0  0.0  1.3889487]
  > Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.0
                 (debye): [0.0  0.0  0.0]  Total: 0.0
 


In [32]:
from qiskit.opflow import X, Z, I
H2_op = (-1.052373245772859 * I ^ I) + \
        (0.39793742484318045 * I ^ Z) + \
        (-0.39793742484318045 * Z ^ I) + \
        (-0.01128010425623538 * Z ^ Z) + \
        (0.18093119978423156 * X ^ X) + (1 * I ^ I) - (1 * I ^ I)
print(H2_op)

-1.052373245772859 * II
+ 0.39793742484318045 * IZ
- 0.39793742484318045 * ZI
- 0.01128010425623538 * ZZ
+ 0.18093119978423156 * XX
+ 1.0 * II
- 1.0 * II
